## 📦 1. Setup & Installation

In [ ]:
# Check GPU
!nvidia-smi

In [ ]:
# Install dependencies
!pip install SimpleITK matplotlib tensorboard scikit-learn tqdm

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Set working directory (adjust path to your Drive)
import os
os.chdir('/content/drive/MyDrive/RADIO_PROJET')
!pwd

## 📊 2. Import Libraries & Setup

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm.notebook import tqdm
import json
from datetime import datetime

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
# Import project modules
import sys
sys.path.append('/content/drive/MyDrive/RADIO_PROJET')

from src.data.dataset import NSCLCDataset
from src.models.unet import UNet, DiceBCELoss

## ⚙️ 3. Configuration

In [ ]:
CONFIG = {
    # Data
    'data_root': 'DATA/processed/normalized_rtstruct',
    'batch_size': 32,
    'num_workers': 2,
    
    # Model
    'in_channels': 1,
    'out_channels': 1,
    
    # Training
    'epochs': 100,
    'learning_rate': 0.001,
    'weight_decay': 1e-5,
    'dice_weight': 0.5,
    
    # Checkpoint
    'save_dir': 'checkpoints_rtstruct',
    'save_freq': 10,
    
    # Device
    'device': 'cuda' if torch.cuda.is_available() else 'cpu'
}

print("Configuration:")
for k, v in CONFIG.items():
    print(f"  {k}: {v}")

## 📁 4. Load Dataset

In [ ]:
print("Loading datasets...")

train_dataset = NSCLCDataset(
    data_root=CONFIG['data_root'],
    split='train',
    slice_wise=True,
    cache_data=False
)

val_dataset = NSCLCDataset(
    data_root=CONFIG['data_root'],
    split='val',
    slice_wise=True,
    cache_data=False
)

print(f"\nTrain: {len(train_dataset)} slices")
print(f"Val: {len(val_dataset)} slices")

In [ ]:
# Visualize sample
sample = train_dataset[1000]
img = sample['image'].squeeze().numpy()
mask = sample['mask'].squeeze().numpy()

fig, axes = plt.subplots(1, 3, figsize=(15, 5))
axes[0].imshow(img, cmap='gray')
axes[0].set_title(f"CT Scan\n{sample['patient_id']} - Slice {sample['slice_idx']}")
axes[0].axis('off')

axes[1].imshow(mask, cmap='Reds', alpha=0.8)
axes[1].set_title(f"Tumor Mask\n{mask.sum():.0f} tumor pixels")
axes[1].axis('off')

axes[2].imshow(img, cmap='gray')
axes[2].imshow(mask, cmap='Reds', alpha=0.5)
axes[2].set_title("Overlay")
axes[2].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Create DataLoaders
train_loader = DataLoader(
    train_dataset,
    batch_size=CONFIG['batch_size'],
    shuffle=True,
    num_workers=CONFIG['num_workers'],
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=CONFIG['batch_size'],
    shuffle=False,
    num_workers=CONFIG['num_workers'],
    pin_memory=True
)

print(f"Train batches: {len(train_loader)}")
print(f"Val batches: {len(val_loader)}")

## 🏗️ 5. Create Model

In [ ]:
device = torch.device(CONFIG['device'])

model = UNet(
    in_channels=CONFIG['in_channels'],
    out_channels=CONFIG['out_channels']
)
model = model.to(device)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

In [ ]:
# Loss and optimizer
criterion = DiceBCELoss(dice_weight=CONFIG['dice_weight'])
optimizer = optim.Adam(
    model.parameters(),
    lr=CONFIG['learning_rate'],
    weight_decay=CONFIG['weight_decay']
)

scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode='max',
    factor=0.5,
    patience=10,
    verbose=True
)

print("Optimizer and scheduler created")

## 🔧 6. Training Functions

In [ ]:
def dice_coefficient(pred, target, smooth=1e-6):
    """Calculate Dice coefficient."""
    pred = (pred > 0.5).float()
    intersection = (pred * target).sum()
    union = pred.sum() + target.sum()
    dice = (2. * intersection + smooth) / (union + smooth)
    return dice.item()

def train_epoch(model, loader, criterion, optimizer, device):
    """Train for one epoch."""
    model.train()
    running_loss = 0.0
    running_dice = 0.0
    
    pbar = tqdm(loader, desc='Training')
    for batch in pbar:
        images = batch['image'].to(device)
        masks = batch['mask'].to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, masks)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        dice = dice_coefficient(outputs, masks)
        running_dice += dice
        
        pbar.set_postfix({'loss': f'{loss.item():.4f}', 'dice': f'{dice:.4f}'})
    
    return running_loss / len(loader), running_dice / len(loader)

def validate(model, loader, criterion, device):
    """Validate the model."""
    model.eval()
    running_loss = 0.0
    running_dice = 0.0
    
    with torch.no_grad():
        pbar = tqdm(loader, desc='Validation')
        for batch in pbar:
            images = batch['image'].to(device)
            masks = batch['mask'].to(device)
            
            outputs = model(images)
            loss = criterion(outputs, masks)
            
            running_loss += loss.item()
            dice = dice_coefficient(outputs, masks)
            running_dice += dice
            
            pbar.set_postfix({'loss': f'{loss.item():.4f}', 'dice': f'{dice:.4f}'})
    
    return running_loss / len(loader), running_dice / len(loader)

## 🚀 7. Training Loop

In [ ]:
# Create checkpoint directory
save_dir = Path(CONFIG['save_dir'])
save_dir.mkdir(parents=True, exist_ok=True)

# Training history
history = {
    'train_loss': [],
    'train_dice': [],
    'val_loss': [],
    'val_dice': []
}

best_dice = 0.0

print("="*70)
print("STARTING TRAINING")
print("="*70)

In [ ]:
# Main training loop
for epoch in range(CONFIG['epochs']):
    print(f"\nEpoch {epoch+1}/{CONFIG['epochs']}")
    print("-" * 70)
    
    # Train
    train_loss, train_dice = train_epoch(model, train_loader, criterion, optimizer, device)
    
    # Validate
    val_loss, val_dice = validate(model, val_loader, criterion, device)
    
    # Update scheduler
    scheduler.step(val_dice)
    
    # Save history
    history['train_loss'].append(train_loss)
    history['train_dice'].append(train_dice)
    history['val_loss'].append(val_loss)
    history['val_dice'].append(val_dice)
    
    # Print summary
    print(f"\nEpoch {epoch+1} Summary:")
    print(f"  Train Loss: {train_loss:.4f} | Train Dice: {train_dice:.4f}")
    print(f"  Val Loss:   {val_loss:.4f} | Val Dice:   {val_dice:.4f}")
    print(f"  LR: {optimizer.param_groups[0]['lr']:.6f}")
    
    # Save checkpoint
    if (epoch + 1) % CONFIG['save_freq'] == 0:
        checkpoint_path = save_dir / f'checkpoint_epoch_{epoch+1:03d}.pth'
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': val_loss,
            'dice': val_dice,
        }, checkpoint_path)
        print(f"  Checkpoint saved: {checkpoint_path.name}")
    
    # Save best model
    if val_dice > best_dice:
        best_dice = val_dice
        best_path = save_dir / 'best_model.pth'
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': val_loss,
            'dice': val_dice,
        }, best_path)
        print(f"  ★ NEW BEST MODEL! Dice: {best_dice:.4f}")

print("\n" + "="*70)
print("TRAINING COMPLETE!")
print("="*70)
print(f"Best Dice Score: {best_dice:.4f}")

## 📈 8. Visualize Training

In [ ]:
# Plot training curves
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Loss
axes[0].plot(history['train_loss'], label='Train', linewidth=2)
axes[0].plot(history['val_loss'], label='Val', linewidth=2)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training & Validation Loss')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Dice
axes[1].plot(history['train_dice'], label='Train', linewidth=2)
axes[1].plot(history['val_dice'], label='Val', linewidth=2)
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Dice Coefficient')
axes[1].set_title('Training & Validation Dice Score')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('training_curves.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"Best validation Dice: {max(history['val_dice']):.4f}")

## 🔍 9. Test Predictions

In [ ]:
# Load best model
checkpoint = torch.load(save_dir / 'best_model.pth')
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

print(f"Loaded best model from epoch {checkpoint['epoch']+1}")
print(f"Best Dice: {checkpoint['dice']:.4f}")

In [ ]:
# Visualize predictions
fig, axes = plt.subplots(4, 4, figsize=(16, 16))
axes = axes.ravel()

with torch.no_grad():
    for i in range(16):
        idx = np.random.randint(0, len(val_dataset))
        sample = val_dataset[idx]
        
        img = sample['image'].unsqueeze(0).to(device)
        mask = sample['mask'].squeeze().numpy()
        
        pred = model(img)
        pred = (pred.squeeze().cpu().numpy() > 0.5).astype(float)
        
        # Show overlay
        img_np = sample['image'].squeeze().numpy()
        axes[i].imshow(img_np, cmap='gray')
        axes[i].imshow(mask, cmap='Greens', alpha=0.3)
        axes[i].imshow(pred, cmap='Reds', alpha=0.3)
        axes[i].set_title(f"{sample['patient_id']}\nSlice {sample['slice_idx']}")
        axes[i].axis('off')

plt.tight_layout()
plt.savefig('predictions_sample.png', dpi=300, bbox_inches='tight')
plt.show()

print("Green: Ground Truth | Red: Prediction")

## 💾 10. Save Results

In [ ]:
# Save training history
import json

with open(save_dir / 'training_history.json', 'w') as f:
    json.dump(history, f, indent=4)

# Save config
with open(save_dir / 'config.json', 'w') as f:
    json.dump(CONFIG, f, indent=4)

print("Results saved!")
print(f"  - Model: {save_dir / 'best_model.pth'}")
print(f"  - History: {save_dir / 'training_history.json'}")
print(f"  - Config: {save_dir / 'config.json'}")